In [7]:

import re
from collections import defaultdict, Counter
from itertools import groupby
from nltk import flatten
from nltk.util import trigrams as get_trigrams

def make_bigram_dict():
    this_dict = defaultdict(Counter)
    return this_dict


In [34]:
trigram_merged_bw

{'A': {'c': Counter({('d', 'a'): 1,
           ('h', 'g'): 1,
           ('a', 'o'): 1,
           ('c', 'a'): 1}),
  'b': Counter({('c', 'd'): 1,
           ('c', 'h'): 1,
           ('c', 'c'): 1,
           ('e', 'b'): 1,
           ('z', 'y'): 1}),
  'e': Counter({('b', 'c'): 1}),
  'f': Counter({('b', 'e'): 1}),
  'g': Counter({('f', 'b'): 1}),
  'h': Counter({('g', 'f'): 1}),
  'i': Counter({('b', 'c'): 1}),
  'j': Counter({('i', 'b'): 1}),
  'k': Counter({('j', 'i'): 1}),
  'a': Counter({('k', 'j'): 1, ('o', 'n'): 1, ('q', 'r'): 1, ('n', 'z'): 1}),
  'y': Counter({('a', 'k'): 1}),
  'z': Counter({('y', 'a'): 1, ('r', 'x'): 1}),
  'n': Counter({('b', 'z'): 1, ('z', 'r'): 1}),
  'o': Counter({('n', 'b'): 1}),
  'p': Counter({('b', 'c'): 1}),
  'q': Counter({('p', 'b'): 1, ('r', 'q'): 1}),
  'r': Counter({('q', 'p'): 1, ('x', 'a'): 1}),
  'x': Counter({('a', 'q'): 1})},
 'B': {'b': Counter({('i', 'y'): 1,
           ('r', 'q'): 1,
           ('c', 'x'): 1,
           ('q', 'p'): 1,

In [ ]:

    while True:
        raw_lines = corpus_file.readlines(10000)
        if not raw_lines:
            break
        org_items = groupby(raw_lines, key=lambda x: re.match(r'(^.)', x).group(1))
        unigrams = {
            key:[clean_bnc_line(line) for line in group] for key, group in org_items
            }
        for corpus, unigram_list in unigrams.items():
            chunk_unigrams = flatten(unigram_list)
            unigram_freqs[corpus].update(chunk_unigrams)
        trigrams = {
            key:[bigram for line in group for bigram in get_trigrams(line) if len(line) > 1]
            for key, group in unigrams.items()
            }
        trigram_counts = {corpus: Counter(trigram_ocs) for corpus, trigram_ocs in trigrams.items()} #dunno if this step is necessary
        for corpus, counts in trigram_counts.items():
            for trigram, freq in counts.items():
                trigram_fw[corpus][trigram[0]][trigram[1]].update([trigram[2]])
                trigram_bw[corpus][trigram[2]][trigram[1]].update([trigram[0]])
        i += 1
        if verbose:
            i += len(raw_lines)
            print(f'{i} lines processed')
print('Merging....')        
trigram_merged_bw = {corpus: {this_c3: Counter({(this_c2, this_c1): freq for this_c2, c1_freqs in c2_freqs.items() for this_c1, freq in c1_freqs.items()})  for this_c3, c2_freqs in c3_freqs.items()} for corpus, c3_freqs in trigram_bw.items()}
n_trigrams = [sum(bigram_freqs.values()) for corpus_freqs in trigram_merged_bw.values() for bigram_freqs in corpus_freqs.values()]
n_trigrams = sum(n_trigrams)
return (unigram_freqs, n_trigrams, trigram_fw, trigram_bw, trigram_merged_bw)
